In [13]:
#coding=utf-8
import xlwings as xw
import re

In [14]:
# 打開活頁簿檔案
file_path = 'hoo-goa-chu-im.xlsx'
wb = xw.Book(file_path)

# 指定來源工作表
source_sheet = wb.sheets['工作表1']

# 取得工作表內總列數
row_num = source_sheet.range('A' + str(wb.sheets[0].cells.last_cell.row)).end('up').row
print(f'row_num = {row_num}')

row_num = 12


In [15]:
#==========================================================
# 備妥程式需使用之工作表
#==========================================================
sheet_name_list = ["字庫表", "缺字表", "漢字注音表", "兩欄對照表"]
#-----------------------------------------------------
# 檢查工作表是否已存在
for sheet_name in sheet_name_list:
    sheet = wb.sheets[sheet_name]
    try:
        sheet.select()
        sheet.clear()
        continue
    except: 
        # CommandError 的 Exception 發生日，表工作表不存在
        # 新增程式需使用之工作表
        wb.sheets.add(name=sheet_name)

tui_chiau_sheet = wb.sheets["漢字注音表"]

In [16]:
#==========================================================
# 主程式
#==========================================================
row_content = []
start_row = 1
end_row = row_num
row = 1
i = 1  # index for source_data sheet

while row < end_row:
    # Read data from source_sheet 
    read_data  = str(source_sheet.range('A' + str(row)).value)
    han_ji = read_data.strip()
    
    # 讀到空白行
    if han_ji == "None":
        row += 1
        continue
    
    # 讀到標點符號
    piau_tiam = han_ji.count('，')
    piau_tiam2 = han_ji.count('；')
    ku_ho = han_ji.count('。')
    if ku_ho == 0:
        un = piau_tiam + piau_tiam2 + 1
    else:
        un = piau_tiam + piau_tiam2 
    row += 1

    chu_im = ""
    for j in range(0, un):
        buffer = str(source_sheet.range('A' + str(row)).value)
        buffer.strip()
        chu_im += (buffer + ' ') 
        row += 1
    row += 1
    
    # Write to target cell
    # Append ku_ho
    if ku_ho == 0:
        han_ji += '。'
    # Strip Line ID
    searchObj = re.search( r'(^\d+.\s)(.+)', han_ji, re.M|re.I)
    if searchObj:
        tui_chiau_sheet.range('A' + str(i)).value = searchObj.group(2)
    else:        
        tui_chiau_sheet.range('A' + str(i)).value = han_ji 
    tui_chiau_sheet.range('B' + str(i)).value = chu_im 
    i += 1